# messingWithBLUNTstrings.ipynb
## Marcus Viscardi,    August 03, 2023

Joshua built a script to create long strings that indicate whether a nucleotide is overlapping with a UTR, CDS, start/stop, or intron/intergenic region.
These strings are currently in a pretty long format (exactly as long as the read) and I want to see if I can condense them down to a more manageable size and push them into a SAM/BAM file for easy usage.

In [2]:
from pathlib import Path
import pandas as pd
import nanoporePipelineCommon as npC
from pprint import pprint
import pysam

print(f"Imports done @ {npC.get_dt(for_print=True)}")

Imports done @ 08/09/23 @ 10:25:11 AM


In [3]:
path_to_working_dir = Path("/data16/joshua/working/230803_psForLM")
path_dict = {}
for pathy in path_to_working_dir.iterdir():
    if pathy.suffix in ['.sam', '.longjam'] and pathy.stem != 'temp':
        treatment = pathy.stem.split("_")[-2]
        path_dict[f"{treatment}{pathy.suffix}"] = pathy
pprint(path_dict)

{'MockTAD.longjam': PosixPath('/data16/joshua/working/230803_psForLM/230724_nanoporeRun_sES-RNAStds_50-50_LT_MockTAD_Nano3P.AtoG.sorted.shorter.longjam'),
 'MockTAD.sam': PosixPath('/data16/joshua/working/230803_psForLM/230724_nanoporeRun_sES-RNAStds_50-50_LT_MockTAD_Nano3P.AtoG.sorted.sam'),
 'TAD.longjam': PosixPath('/data16/joshua/working/230803_psForLM/230724_nanoporeRun_sES-RNAStds_50-50_LT_TAD_Nano3P.AtoG.sorted.shorter.longjam'),
 'TAD.sam': PosixPath('/data16/joshua/working/230803_psForLM/230724_nanoporeRun_sES-RNAStds_50-50_LT_TAD_Nano3P.AtoG.sorted.sam')}


In [4]:
df = pd.read_csv(path_dict['MockTAD.longjam'], header=None, sep='\t', skiprows=14, names=list(range(24)))
df

/tmp/ipykernel_3936946/2354929855.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_dict['MockTAD.longjam'], header=None, sep='\t', skiprows=14, names=list(range(24)))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,151d3163-c360-46f3-a678-ed12855bfb0e,0.0,I,16516.0,56.0,2S21M1I13M4D5M5D3M4D19M2D14M1I17M5D24M1D10M1D5...,*,0.0,0.0,ATCGGAGAACGCCTCGGAGTCGGCCCCGGAGGGGGTTCGTATTGCA...,*,NM:i:51,ms:i:256,AS:i:256,nn:i:0,tp:A:P,cm:i:10,s1:i:76,s2:i:0,de:f:0.0882,MD:Z:34^TTGG2G2^CTTCG3^TCGT19^TC31^GCGGC2G21^C...,rl:i:0,NaN,NaN
1,CCCCCCCCCCCCCCCCCCCCCaCCCCCCCCCCCCCCCCCCCCCCCC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d7345efd-76da-4a04-a3e1-6422c6e5083d,0.0,I,16609.0,60.0,2S69M3D14M3D7M1D12M3D56M1I8M1D57M4D40M1I4M1D3M...,*,0.0,0.0,TCAAATTGTCCATCTGACAACTGGGATTTCGGTTCTTCTTGGCTTC...,*,NM:i:64,ms:i:354,AS:i:354,nn:i:0,tp:A:P,cm:i:25,s1:i:158,s2:i:0,de:f:0.1347,MD:Z:59G1T7^CCC2G2C0T0C6^GGT0T0T3T1^T0T11^CCT1...,rl:i:128,NaN,NaN
3,UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,149887a4-0e4d-44db-b95b-4dec334299ed,0.0,I,16629.0,41.0,4S16M1I6M5D7M1D13M4D19M1D16M1I3M5D15M3I44M2D45...,*,0.0,0.0,ATTCAAAATTTCGGTTCTTCTTTGGCTTATTTGTGGGTGGTTTATT...,*,NM:i:62,ms:i:334,AS:i:334,nn:i:0,tp:A:P,cm:i:14,s1:i:101,s2:i:0,de:f:0.1073,MD:Z:22^TCTTC7^G13^TCCC19^G19^TTTTT59^GT59^TTT...,rl:i:146,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498657,UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498658,62d7c56d-d4f8-4a97-8367-a3ecb673f505,0.0,X,17680679.0,60.0,5S88M1I27M6D25M115S,*,0.0,0.0,GGAATTGCCCCTGTGTTGGTTTTCATCTTGTGGGTTTTGTCGTTTT...,*,NM:i:11,ms:i:234,AS:i:234,nn:i:0,tp:A:P,cm:i:14,s1:i:102,s2:i:0,de:f:0.0423,MD:Z:92C22^TCGTTT5C1T6C10,rl:i:76,NaN,NaN
498659,UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498660,5833caab-0885-4307-99a1-bb803538d9ac,0.0,X,17718433.0,60.0,107S71M1D49M63S,*,0.0,0.0,TACACCCTCCCCGCAATCACTGCCACTCTGACCTTGCCGCCTACAA...,*,NM:i:2,ms:i:228,AS:i:228,nn:i:0,tp:A:P,cm:i:15,s1:i:99,s2:i:0,de:f:0.0165,"SA:Z:X,17717329,+,1S108M7D181S,34,11;",MD:Z:11C59^C49,rl:i:0,NaN


In [22]:
def compress_blunt(blunt_string: str) -> str:
    compressed = ""
    count = 1

    for i in range(1, len(blunt_string)):
        if blunt_string[i] == blunt_string[i - 1]:
            count += 1
        else:
            compressed += str(count) + blunt_string[i - 1]
            count = 1

    # Add the last character and its count
    compressed += str(count) + blunt_string[-1]

    return compressed

blunt_series = df.loc[1::2, 0].apply(lambda x: compress_blunt(x)).rename('blunt_string')
read_name_series = df.loc[::2, 0].reset_index(drop=True).rename('read_name')
blunt_series.reset_index(drop=True, inplace=True)
read_blunt_df = pd.concat([read_name_series, blunt_series], axis=1)
read_blunt_df = read_blunt_df.set_index('read_name')
read_blunt_df.loc['151d3163-c360-46f3-a678-ed12855bfb0e', 'blunt_string']

0


'21C1a33C1T20U1a146U'

In [23]:
# Now I want to loop through the original sam file and add the compressed blunt string as a tag to each read and write to a new sam
sam = pysam.AlignmentFile(path_dict['MockTAD.sam'], "rb")
# new_sam_path = path_dict['MockTAD.sam'].parent / f"{path_dict['MockTAD.sam'].stem}_re-compressed.sam"
new_sam_path = Path("/tmp") / f"{path_dict['MockTAD.sam'].stem}_re-compressed.sam"
new_sam = pysam.AlignmentFile(new_sam_path, "w", template=sam)
for read in sam:
    try:
        read.set_tag("bL", read_blunt_df.loc[read.query_name, 'blunt_string'])
    except KeyError:
        read.set_tag("bL", "NA")
    new_sam.write(read)